# Example of multithreading in Julia

In this example we compute the complex correlation coefficient of two velocity time series (u1,v1) and (u2,v2). We use this example to improve the performance of julia code

In [1]:
using Base.Threads
using BenchmarkTools
using Test

Following the instructions [here](https://github.com/gher-uliege/Documentation/wiki/Julia-Jupyter-notebook-with-multiple-threads) to create a julia kernel multiple threads.


`Threads.nthreads()` should not be 1.

In [2]:
@show Threads.nthreads()

Threads.nthreads() = 8


8

A "native" implementation to compute the complex correlation coefficient of two velocity time series (u1,v1) and (u2,v2) following the vectorized approach from numpy and matlab.

In [3]:
function complex_correlation(u1,v1,u2,v2)
    w1 = u1 + im * v1;
    w2 = u2 + im * v2;
    
    gamma = w1' * w2 / sqrt( (w1'*w1) * (w2'*w2) );
    mag = abs(gamma);
    phi = angle(gamma)*180/pi;
    return mag, phi
end


complex_correlation (generic function with 1 method)

Let's check that `complex_correlation` is correct. A vector time series should correlate perfectly with itself:

In [4]:
u1 = randn(100);
v1 = randn(100);
u2 = u1
v2 = v1
mag, phi = complex_correlation(u1,v1,u2,v2)

@test mag ≈ 1
@test phi ≈ 0

Test Passed

If we apply a rotation of 20° (https://en.wikipedia.org/wiki/Rotation_matrix)

In [5]:
u1 = randn(100);
v1 = randn(100);
u2 = u1 * cosd(20) - v1 * sind(20);
v2 = u1 * sind(20) + v1 * cosd(20);
mag, phi = complex_correlation(u1,v1,u2,v2)

@test mag ≈ 1
@test phi ≈ 20

Test Passed

We still have a perfect correlation magnitude of 1 and we recover the rotation of 20°.

Make a large random dataset:

In [6]:
u1 = randn(100,100,1000);
v1 = randn(100,100,1000);
u2 = randn(100,100,1000);
v2 = randn(100,100,1000);
mag = zeros(size(u1)[1:2]);
phi = zeros(size(u1)[1:2]);

@time for j = 1:size(u1,2)
  for i = 1:size(u1,1)
     mag[i,j],phi[i,j] = complex_correlation(u1[i,j,:],v1[i,j,:],u2[i,j,:],v2[i,j,:])
    end
end

  0.442955 seconds (466.66 k allocations: 929.865 MiB, 16.71% gc time, 20.07% compilation time)


For performance critical code, we should wrap the loops inside a function. Also the macro `@btime` is better for performance measurements:

In [7]:
function compute_complex_correlation!(mag,phi,u1,v1,u2,v2)
   for j = 1:size(u1,2)
     for i = 1:size(u1,1)
         mag[i,j],phi[i,j] = complex_correlation(u1[i,j,:],v1[i,j,:],u2[i,j,:],v2[i,j,:])
     end
   end
end

compute_complex_correlation! (generic function with 1 method)

In [8]:
 @btime compute_complex_correlation!(mag,phi,u1,v1,u2,v2)

  221.507 ms (240000 allocations: 921.02 MiB)


This simple change already improved quite a bit the runtime.
Let's also check that the code is type stable by using `@code_warntype`:

In [9]:
#@code_warntype complex_correlation(u1[1,1,:],v1[1,1,:],u2[1,1,:],v2[1,1,:]);

In [10]:
#@code_warntype compute_complex_correlation!(mag,phi,u1,v1,u2,v2);

OK, the code is type stable: all types are concrete (like `Int64`, `Vector{Float64}` shown in blue, or small unions like `Union{Nothing, Tuple{Int64, Int64}}` shown in yellow. There is no `Any` that would be shown in red. 

Note that we have a lot of allocation that we can reduce using `@view`:

In [18]:
function compute_complex_correlation!(mag,phi,u1,v1,u2,v2)
   for j = 1:size(u1,2)
     for i = 1:size(u1,1)
         mag[i,j],phi[i,j] = complex_correlation((@view u1[i,j,:]),(@view v1[i,j,:]),(@view u2[i,j,:]),(@view v2[i,j,:]))
     end
   end
end

compute_complex_correlation! (generic function with 1 method)

In [12]:
@btime compute_complex_correlation!(mag,phi,u1,v1,u2,v2)

  177.958 ms (120000 allocations: 613.10 MiB)


In fact, also `complex_correlation` does a lot of allocations (when computing a new vector `u1 + im * v2` for example) that can be avoided by making an explicit loop:

In [20]:
function complex_correlation(u1,v1,u2,v2)
    w1w2 = 0.0 + im * 0.0
    w1w1 = 0.0
    w2w2 = 0.0    
    
    for n = 1:length(u1)
        w1 = u1[n] + im * v1[n]
        w2 = u2[n] + im * v2[n]
        w1w2 += w1' * w2
        w1w1 += w1' * w1 # or abs2(w1)
        w2w2 += w2' * w2
    end
    
    gamma = w1w2 / sqrt(w1w1 * w2w2);
    mag = abs(gamma);
    phi = angle(gamma)*180/pi;
    return mag, phi
end

complex_correlation (generic function with 1 method)

Optimization can introduce bugs, so lets check that the results is still ok:

In [21]:

u1 = randn(100);
v1 = randn(100);
u2 = u1 * cosd(20) - v1 * sind(20);
v2 = u1 * sind(20) + v1 * cosd(20);
mag, phi = complex_correlation(u1,v1,u2,v2)

@test mag ≈ 1
@test phi ≈ 20

Test Passed

In [22]:
u1 = randn(100,100,1000);
v1 = randn(100,100,1000);
u2 = randn(100,100,1000);
v2 = randn(100,100,1000);
mag = zeros(size(u1)[1:2]);
phi = zeros(size(u1)[1:2]);
@btime compute_complex_correlation!(mag,phi,u1,v1,u2,v2)

  114.653 ms (0 allocations: 0 bytes)


Great! No allocations anymore. Sometimes it helps to disable the bounds checking by using `@inbounds` or use `abs2(w1)` instead of `w1' * w1` (but not here in my tests). Now the outer loop is parallelized using `Threads.@threads

In [23]:
function compute_complex_correlation!(mag,phi,u1,v1,u2,v2)
   Threads.@threads for j = 1:size(u1,2)
     for i = 1:size(u1,1)
         mag[i,j],phi[i,j] = complex_correlation((@view u1[i,j,:]),(@view v1[i,j,:]),(@view u2[i,j,:]),(@view v2[i,j,:]))
     end
   end
end

compute_complex_correlation! (generic function with 1 method)

In [25]:
@btime compute_complex_correlation!(mag,phi,u1,v1,u2,v2)

  22.378 ms (58 allocations: 4.50 KiB)


So we went down from ~420 ms to ~20 ms! Note that multithreading also introduce synchronization overhead.